<hr style="border:2px solid #0176DE"> </hr>
<center><h1 style="color:#173F8A;"> Escuela de Verano en Metodos Iterativos</h1></center> 
<center><h1 style="color:#173F8A;"> EMI 2024 - CMM Chile</h1></center>
<hr style="border:2px solid #0176DE"> </hr>
<h3 style="color:#173F8A;text-align:right;"> Profesores: &nbsp;Nicolás Barnafi<br>Manuel A. Sánchez<br></h3>

<h3 style="color:#03122E;text-align:right;"> 
    Centro de Modelamiento Matematico <br> 
    Instituto de Ingenieria Matematica y Computacional - IMC UC<br>  
</h3>

<hr style="border:2px solid #03122E"> </hr>
<center><h1 style="color:#173F8A;"> Modulo 3: MinRes, GMRES - NGSolve</h1></center> 
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC:
Primaria: 
celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 
Secundaria
gris oscuro: #707070
-->

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.la import EigenValues_Preconditioner
import matplotlib.pyplot as plt
from ngsolve.krylovspace import MinResSolver

In [2]:
help(MinResSolver)

Help on class MinResSolver in module ngsolve.krylovspace:

class MinResSolver(LinearSolver)
 |  MinResSolver(*args, **kwargs)
 |  
 |  #Source: Michael Kolmbauer https://www.numa.uni-linz.ac.at/Teaching/PhD/Finished/kolmbauer-diss.pdf
 |  
 |  Method resolution order:
 |      MinResSolver
 |      LinearSolver
 |      ngsolve.la.BaseMatrix
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: ngsolve.la.BaseMatrix) -> None
 |      
 |      2. __init__(self: ngsolve.la.BaseMatrix, arg0: ngsolve.la.BaseVector) -> None
 |      
 |      3. __init__(self: ngsolve.la.BaseMatrix, arg0: ngsolve.la.MultiVector) -> None
 |      
 |      4. __init__(self: ngsolve.la.BaseMatrix, arg0: ngsolve.bla.MatrixD) -> None
 |      
 |      5. __init__(self: ngsolve.la.BaseMatrix, arg0: object) -> None
 |  
 |  -------------------

## Model problem
We will test the algorithm in this notebook with the solution of the following Poisson problem

\begin{equation}
-\Delta u = 1, \quad \text{in } (0,1)^{2}, \qquad u=0, \quad \text{sobre } x=0 \text{ e }y=0.
\end{equation}

In [3]:
# Ejemplo de la solucion
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
fes = H1(mesh, order=1, dirichlet='left|bottom')
u, v = fes.TnT()
a = BilinearForm(InnerProduct(grad(u), grad(v))*dx).Assemble()
b = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs())*b.vec
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

## Set Poisson 

In [4]:
def SetPoisson(mesh, f=1, p=1, dirichlet_bndry=None, condense=False):
    # H1-conforming Finite Element Space
    fes = H1(mesh, order=p, dirichlet=dirichlet_bndry)
    # Trial and Test Functions
    u, v = fes.TnT()
    # Bilinear form 
    a = BilinearForm(fes, symmetric=True, condense=condense)
    a += (grad(u)*grad(v))*dx 
    # Linear form
    b = LinearForm(f*v*dx)
    # Gridfunction
    gfu = GridFunction(fes)
    return fes, a, b, gfu

## Direct solver

In [5]:
def SolvePoissonDirect(a, b, gfu, fes, condense=False):
    a.Assemble()
    b.Assemble()
    # Direct solver
    inv = a.mat.Inverse(freedofs=fes.FreeDofs())
    # Solve steps depend on condense
    if condense:
        f.vec.data += a.harmonic_extension_trans * b.vec
        gfu.vec.data = inv * b.vec
        gfu.vec.data += a.harmonic_extension * gfu.vec
        gfu.vec.data += a.inner_solve * b.vec
    else: 
        gfu.vec.data = inv * b.vec
    return gfu, fes.ndof
       

## Iterative MinRes, GMRES (no preconditioner)

In [55]:
def SolvePoissonIterative(a, b, gfu, fes, itermethod='CG', condense=False, compute_condnum=False):
    a.Assemble()
    b.Assemble()
    preI = Projector(mask=fes.FreeDofs(), range=True)
    # Solve
    if itermethod=='CG':
        solvers.CG(mat =a.mat, 
                      rhs = b.vec, 
                      pre=None, freedofs=fes.FreeDofs(), 
                      sol =gfu.vec, 
                      tol=1e-8,
                      maxsteps=10000, 
                      printrates='\r')
    elif itermethod=='GMRes':
        solvers.GMRes(A =a.mat, 
                      b = b.vec, 
                      pre=None, freedofs=fes.FreeDofs(), 
                      x =gfu.vec, 
                      tol=1e-8,
                      maxsteps=10000, 
                      printrates='\r')
    elif itermethod=='MinRes':
        solvers.MinRes(mat =a.mat, 
                      rhs = b.vec, 
                      pre=preI, #freedofs=fes.FreeDofs(), 
                      sol =gfu.vec, 
                      tol=1e-8,
                      maxsteps=10000, 
                      printrates='\r')
    if compute_condnum is False:
        return gfu, ()
    else:
        lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
        kappa = max(lams)/min(lams)
        return gfu, kappa
def SolvePoissonIterativeGMRes(a, b, gfu, fes, condense=False, compute_condnum=False):
    a.Assemble()
    b.Assemble()
    preI = Projector(mask=fes.FreeDofs(), range=True)
    inv = GMRESSolver(a.mat, pre=preI, maxsteps=10000)
    # Solve steps depend on condense
    if condense:
        f.vec.data += a.harmonic_extension_trans * b.vec
        gfu.vec.data = inv * b.vec
        gfu.vec.data += a.harmonic_extension * gfu.vec
        gfu.vec.data += a.inner_solve * b.vec
    else: 
        gfu.vec.data = inv * b.vec
    if compute_condnum is False:
        return gfu, inv.GetSteps()
    else:
        lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
        kappa = max(lams)/min(lams)
        return gfu, ((inv.GetSteps(), kappa))

In [56]:
def SolvePoissonIterativeGMRes2(a, b, gfu, fes, condense=False, compute_condnum=False):
    a.Assemble()
    b.Assemble()
    preI = Projector(mask=fes.FreeDofs(), range=True)
    # Solve
    solvers.GMRes(A =a.mat, 
                  b = b.vec, 
                  pre=None, freedofs=fes.FreeDofs(), 
                  x =gfu.vec, 
                  tol=1e-8,
                  maxsteps=10000, 
                  printrates='\r')
    if compute_condnum is False:
        return gfu, ()
    else:
        lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
        kappa = max(lams)/min(lams)
        return gfu, kappa

In [64]:
# parametros
param = dict()
param['h'] = 0.05
param['f'] = 1
param['p'] = 1
param['condense'] = False
mesh = Mesh(unit_square.GenerateMesh(maxh=param['h']))
fes, a, b, gfu = SetPoisson(mesh, p=param['p'], f=param['f'], dirichlet_bndry='bottom|left', condense=param['condense'])

In [65]:
%%time
# gfu_GMRes, steps = SolvePoissonIterativeGMRes(a, b, gfu, fes, condense=False,compute_condnum=True)
# gfu_GMRes, kappa = SolvePoissonIterativeGMRes2(a, b, gfu, fes, condense=False,compute_condnum=False)
gfu_CG, kappa1 = SolvePoissonIterative(a, b, gfu, fes, itermethod='CG', condense=False,compute_condnum=True)
gfu_GMRes, kappa2 = SolvePoissonIterative(a, b, gfu, fes, itermethod='GMRes', condense=False,compute_condnum=True)
gfu_MinRes, kappa3 = SolvePoissonIterative(a, b, gfu, fes, itermethod='MinRes', condense=False,compute_condnum=True)

print(f"condition number of the system: {kappa1, kappa2, kappa3}")

CG converged in 86 iterations to residual 3.385502685189542e-10
GMRes converged in 77 iterations to residual 9.492291226890242e-09
LinearSolver converged in 85 iterations to residual 3.492260823777903e-10
condition number of the system: (543.534539695404, 547.3051169574469, 547.3246228263965)
Wall time: 191 ms


In [59]:
Draw(gfu_CG)
Draw(gfu_GMRes)
Draw(gfu_MinRes)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [20]:
# from time import time
# h0 = 0.05
# levels = 5
# def SolvePoissonIterativeGMRes_levels(h=h0, levels=levels, condense=False, compute_condnum=False):
#     mesh = Mesh(unit_square.GenerateMesh(maxh=h))
#     fes, a, b, gfu = SetPoisson(mesh, dirichlet_bndry='bottom|left')
#     tiempo = []
#     steps = []
#     for l in range(levels):
#         if l > 0: mesh.Refine()
#         fes.Update()
#         gfu.Update()
#         with TaskManager():
#             start = time()
#             gfu, invsteps = SolvePoissonIterativeGMRes(a, b, gfu, fes, condense=condense, compute_condnum=compute_condnum)
#             print (f"ndof = {fes.ndof}, time = {time()-start}, steps = {invsteps}")
#             tiempo.append ( (fes.ndof, time()-start ))
#         steps.append ( (fes.ndof, invsteps ))
#     return gfu, tiempo, steps

# _, time_cg, steps = SolvePoissonIterativeGMRes_levels(h=h0, levels=levels, condense=False, compute_condnum=False)

In [21]:
from time import time
h0 = 0.05
levels = 5
def SolvePoissonIterativeGMRes2_levels(h=h0, levels=levels, condense=False, compute_condnum=False):
    mesh = Mesh(unit_square.GenerateMesh(maxh=h))
    fes, a, b, gfu = SetPoisson(mesh, dirichlet_bndry='bottom|left')
    tiempo = []
    steps = []
    for l in range(levels):
        if l > 0: mesh.Refine()
        fes.Update()
        gfu.Update()
        with TaskManager():
            start = time()
            gfu, kappa = SolvePoissonIterativeGMRes2(a, b, gfu, fes,  condense=condense, compute_condnum=compute_condnum)
            print (f"ndof = {fes.ndof}, time = {time()-start}, steps = {kappa}")
            tiempo.append ( (fes.ndof, time()-start ))
        steps.append ( (fes.ndof, kappa ))
    return gfu, tiempo, kappa

_, time_cg, kappa = SolvePoissonIterativeGMRes2_levels(h=h0, levels=levels, condense=False, compute_condnum=False)

GMRes converged in 77 iterations to residual 9.492291226890242e-09
ndof = 508, time = 0.22079753875732422, steps = ()
GMRes converged in 189 iterations to residual 9.40615247100902e-09
ndof = 1949, time = 1.2626373767852783, steps = ()
GMRes converged in 361 iterations to residual 9.908136985843355e-09
ndof = 7633, time = 5.3175742626190186, steps = ()
GMRes converged in 678 iterations to residual 9.997995817401705e-09
ndof = 30209, time = 25.673316478729248, steps = ()
GMRes converged in 1248 iterations to residual 9.97846339744554e-09
ndof = 120193, time = 158.08091568946838, steps = ()


In [ ]:
help(solvers.CG)

In [ ]:
help(solvers.GMRes)

In [ ]:
help(CGSolver)

In [ ]:
help(la)

In [ ]:
help(GMRESSolver)
help(QMRSolver)
help(MINRESSolver)

In [ ]:
help(krylovspace)

In [ ]:
help(krylovspace.MinResSolver)

In [77]:
# Mesh
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))

In [96]:
condense=True
mesh = Mesh(unit_square.GenerateMesh(maxh=0.5))
fes = H1(mesh, order=1, dirichlet='left|bottom')
u, v = fes.TnT()
a = BilinearForm(InnerProduct(grad(u), grad(v))*dx, condense=condense)
b = LinearForm(1*v*dx)
gfu = GridFunction(fes)

In [97]:

a.Assemble()
b.Assemble()
# Direct solver
inv = a.mat.Inverse(freedofs=fes.FreeDofs(coupling=condense))
# Solve steps depend on condense
if condense:
    b.vec.data += a.harmonic_extension_trans * b.vec
gfu.vec.data = inv * b.vec
if condense:
    gfu.vec.data += a.harmonic_extension * gfu.vec
    gfu.vec.data += a.inner_solve * b.vec

In [98]:
condense=True
for l in range(1):
    if l > 0:
        mesh.Refine()
    fes.Update()
    gfu.Update()
    with TaskManager():
        a.Assemble()
        b.Assemble()
        # Direct solver
        inv = a.mat.Inverse(freedofs=fes.FreeDofs(coupling=condense))
        # Solve steps depend on condense
        if condense:
            b.vec.data += a.harmonic_extension_trans * b.vec
        gfu.vec.data = inv * b.vec
        if condense:
            gfu.vec.data += a.harmonic_extension * gfu.vec
            gfu.vec.data += a.inner_solve * b.vec

In [80]:
Draw(gfu)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene